In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('listings.csv', header=0, index_col=0)

In [3]:
# Select columns

selected_cols = ['zipcode', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 
                'amenities', 'price', 'minimum_nights', 'cancellation_policy']
data = data[selected_cols]

In [4]:
# Create dummy variables

categorical_variable = ['property_type', 'room_type', 'cancellation_policy']
temp = pd.get_dummies(data[categorical_variable])
data = pd.concat([data, temp], axis=1).drop(categorical_variable, axis=1)

In [5]:
# Convert price format

data['price'] = (data['price'].str.replace(r'[\$,]', '').astype(float))

In [6]:
# Drop any entries that are inconsistent; i.e. 'accommodates', 'bedrooms', 'beds', or 'price' with a value of 0

data = data[data['accommodates'] != 0]
data = data[data['bedrooms'] != 0]
data = data[data['beds'] != 0]
data = data[data['price'] != 0]

In [7]:
# Handle invalid zipcodes

data['zipcode'] = pd.to_numeric(data['zipcode'], errors='coerce')
data = data.dropna()

In [8]:
# Create feature vectors

from collections import Counter

amenities = data['amenities']
x = []
for i in range(len(amenities)):
    row = amenities.iloc[i][1: -1].split(',')
    for item in row:
        item = item.replace('"', '')
        x.append(item)

c = Counter(x)
del c['']

In [9]:
all_amenities = list(c.keys())

In [10]:
for i in range(len(amenities)):
    row = amenities.iloc[i][1: -1].split(',')
    for item in row:
        item = item.replace('"', '')
        if item not in data.columns:
            data[item] = 0
        data.set_value(data.index[i], item, 1)

In [11]:
data = data.drop('amenities', axis=1)
data.head()

,zipcode,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,...,Suitable for events,Self Check-In,Keypad,Lockbox,Other pet(s),,Free parking on street,Doorman Entry,Washer / Dryer,Paid parking off premises
id,,,,,,,,,,,,,,,,,,,,,
685006,11225.0,2,1.0,1.0,1.0,75.0,2,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
9461238,11211.0,5,1.0,1.0,2.0,117.0,2,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4873690,11101.0,2,1.0,1.0,1.0,89.0,1,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
12738047,10025.0,2,1.0,1.0,1.0,80.0,5,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
15359479,10003.0,4,1.0,1.0,1.0,200.0,5,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
data['Y'] = data['price']
data.pop('price')
data.head()

,zipcode,accommodates,bathrooms,bedrooms,beds,minimum_nights,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,property_type_Boutique hotel,...,Self Check-In,Keypad,Lockbox,Other pet(s),,Free parking on street,Doorman Entry,Washer / Dryer,Paid parking off premises,Y
id,,,,,,,,,,,,,,,,,,,,,
685006,11225.0,2,1.0,1.0,1.0,2,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,75.0
9461238,11211.0,5,1.0,1.0,2.0,2,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,117.0
4873690,11101.0,2,1.0,1.0,1.0,1,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,89.0
12738047,10025.0,2,1.0,1.0,1.0,5,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,80.0
15359479,10003.0,4,1.0,1.0,1.0,5,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,200.0


In [13]:
del data['']
del data.index.name

In [14]:
data.to_csv('airbnb_v1.csv')
data.shape

(36096, 89)